How It Works?

[User Input] --> [Investigator Agent] --(Tool: Search)--> [Raw Data]
                        |
                  [Analyst Agent] --(Logic)--> [Key Insights]
                        |
                  [Reporter Agent] --> [Final Report]

In [ ]:
# Installing the Google Generative AI SDK
# I'm using duckduckgo_search as my search tool because it requires no API key for this demo,
# making the agent easier to deploy and test.
!pip install -q -U google-generativeai duckduckgo-search markdown

Importing Dependencies

In [ ]:
import os
import google.generativeai as genai
from duckduckgo_search import DDGS
from IPython.display import display, Markdown, HTML

# CONFIGURATION
# I'm using Gemini 1.5 Flash because it's fast and cheap for iterative agent tasks
GEMINI_API_KEY = "YOUR_API_KEY_HERE" # PASTE YOUR KEY HERE
genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel('gemini-1.5-flash')

print("Setup Complete. Ready to build agents.")

The Tool Layer

In [ ]:
class SearchTool:
    """
    A specific tool for the agents to access the internet.
    I wrapped this in a class to make it modular if I want to swap search providers later.
    """
    def search(self, query, max_results=5):
        print(f"   [Tool Log] Searching for: {query}...")
        results = []
        try:
            with DDGS() as ddgs:
                # Searching for text results
                search_gen = ddgs.text(query, max_results=max_results)
                for r in search_gen:
                    results.append(f"Title: {r['title']}\nURL: {r['href']}\nSnippet: {r['body']}")
        except Exception as e:
            return f"Error during search: {e}"

        return "\n---\n".join(results)

# Initialize the tool
search_tool = SearchTool()

The Agent Framework

In [ ]:
class Agent:
    """
    Base class for my agents.
    It holds the model, the system instructions (persona), and the logic to execute tasks.
    """
    def __init__(self, name, role, model):
        self.name = name
        self.role = role
        self.model = model

    def perform_task(self, context, task_prompt):
        """
        Sends the context and task to Gemini.
        """
        print(f"\n🤖 {self.name} is working...")

        # Context Engineering: Compacting previous findings into the prompt
        full_prompt = f"""
        You are {self.name}. Your role is: {self.role}.

        Current Project Context:
        {context}

        YOUR TASK:
        {task_prompt}
        """

        response = self.model.generate_content(full_prompt)
        return response.text

Defining Specific Agents

In [ ]:
# 1. The Investigator: Finds the data
investigator = Agent(
    name="Investigator Agent",
    role="You are an expert OSINT researcher. Your job is to generate search queries and parse raw data. You do not assume; you only find.",
    model=model
)

# 2. The Analyst: Makes sense of the data
analyst = Agent(
    name="Analyst Agent",
    role="You are a senior risk analyst. You look for red flags, business models, and reputation issues. You are skeptical and thorough.",
    model=model
)

# 3. The Reporter: Formats it for humans
reporter = Agent(
    name="Reporter Agent",
    role="You are an executive communications officer. You take complex analysis and turn it into clean, readable Markdown reports.",
    model=model
)

The Orchestration Logic

In [ ]:
def run_investigation(target_entity):
    print(f"🚀 Starting DeepDive investigation on: {target_entity}")
    print("-" * 50)

    # State Management: This dictionary holds our accumulated knowledge
    investigation_state = {
        "target": target_entity,
        "raw_search_data": "",
        "analysis": ""
    }

    # STEP 1: RESEARCH
    # First, we ask Gemini to generate good search queries based on the target
    query_gen_prompt = f"Generate 3 specific search queries to find the background, controversies, and business model of '{target_entity}'. Return ONLY the queries separated by commas."
    queries = investigator.perform_task("Target: " + target_entity, query_gen_prompt)
    clean_queries = [q.strip() for q in queries.split(',')]

    # Execute the searches using the Tool
    full_search_dump = ""
    for q in clean_queries:
        full_search_dump += search_tool.search(q) + "\n"

    investigation_state["raw_search_data"] = full_search_dump

    # STEP 2: ANALYSIS
    # Pass the raw search data to the Analyst
    analysis_prompt = f"""
    Analyze the following raw search results for '{target_entity}'.
    Identify:
    1. What they actually do.
    2. Any negative sentiment or controversies.
    3. Key leadership or figures associated.
    4. Recent major news.

    Raw Data:
    {investigation_state['raw_search_data']}
    """
    investigation_state["analysis"] = analyst.perform_task(investigation_state["raw_search_data"], analysis_prompt)

    # STEP 3: REPORTING
    # Pass the analysis to the Reporter for final formatting
    report_prompt = f"""
    Create a professional 'Due Diligence Dossier' based on this analysis.
    Use Markdown formatting. Include headers, bullet points, and a 'Risk Level' assessment at the top.

    Analysis to format:
    {investigation_state["analysis"]}
    """
    final_report = reporter.perform_task(investigation_state["analysis"], report_prompt)

    return final_report

Execution

In [ ]:
# --- USER INPUT ---
target = "Theranos" # Change this to any company or person to test
# ------------------

result = run_investigation(target)

print("\n\n" + "="*30)
print("FINAL DOSSIER GENERATED")
print("="*30)
display(Markdown(result))